# TFIM with NetKet: From 1D and 2D Baselines to Long-Range Interactions


This notebook is the full blog post, with narrative, theory, and executable code.
Run cells top-to-bottom to reproduce the results.


In [ ]:
import os
os.environ.setdefault("JAX_PLATFORM_NAME", "cpu")
os.environ.setdefault("JAX_ENABLE_X64", "0")

# Force CPU to avoid Metal backend dtype limitations.


This post is a practical, minimal-yet-complete introduction to simulating the transverse-field Ising model (TFIM) with NetKet. We start by reproducing reference energies in 1D and 2D, then add long-range interactions and compare to exact diagonalization (ED) on small sizes, before scaling up to larger lattices. The code snippets are intentionally small and clean; each block is meant to be copied and run as-is with minor parameter tweaks.

## Goal and workflow
We start with small, verifiable systems to catch implementation and sampling issues early. ED is the gold standard for small sizes, so it anchors our VMC results. Long-range couplings probe whether the variational ansatz can handle nonlocal correlations. Scaling then tests how accuracy and cost change with system size and ansatz capacity.


We will follow a simple, repeatable workflow:

1. **Baseline checks (1D and 2D):** reproduce ground-state energies in known regimes.
2. **Long-range interactions:** introduce power-law couplings and validate against ED for small sizes.
3. **Scaling:** increase system size and evaluate how accuracy, variance, and runtime scale.
4. **Open questions:** note where physics or numerics might become interesting.

## Prerequisites

- Python environment with NetKet installed.
- Basic familiarity with TFIM and variational Monte Carlo (VMC).

A minimal environment (using `uv`):



In [ ]:
!uv venv .venv
!uv pip install --python .venv/bin/python netket
!source .venv/bin/activate



## Model definition: TFIM on a graph
The TFIM interpolates between an ordered Ising phase (small h/J) and a quantum paramagnet (large h/J). In 1D, the critical point is at h/J = 1, so sweeping h/J is a standard diagnostic. We encode the lattice as a graph and let NetKet build the operator from that structure.


The TFIM Hamiltonian is:

\[
H = -J \sum_{\langle i,j \rangle} \sigma_i^z \sigma_j^z - h \sum_i \sigma_i^x
\]

In NetKet, we define a graph (chain or grid), a spin Hilbert space, and the Ising operator. The following helper keeps the setup minimal.



In [ ]:
import netket as nk


def make_tfim(graph, J=1.0, h=1.0):
    hilbert = nk.hilbert.Spin(s=0.5, N=graph.n_nodes)
    H = nk.operator.Ising(hilbert=hilbert, graph=graph, J=J, h=h)
    return hilbert, H



## Baseline 1D: reproduce ground-state energy
This is a sanity check: VMC should approach the exact ground-state energy for small N. If this fails, either the operator is wrong, sampling is unstable, or the optimization is misconfigured.


Start with a small 1D chain to verify the VMC machinery. Use ED for the smallest sizes.



In [ ]:
import netket as nk

N = 12
J = 1.0
h = 1.0

chain = nk.graph.Chain(length=N, pbc=True)
hilbert, H = make_tfim(chain, J=J, h=h)

# Exact diagonalization (small N)
E_ed = nk.exact.lanczos_ed(H, k=1, compute_eigenvectors=False)[0]
print("ED ground-state energy:", E_ed)

# VMC setup
model = nk.models.RBM(alpha=2)
sa = nk.sampler.MetropolisLocal(hilbert, n_chains=32)
vstate = nk.vqs.MCState(sa, model, n_samples=2048, n_discard_per_chain=100)

opt = nk.optimizer.Sgd(learning_rate=0.05)
vmc = nk.VMC(hamiltonian=H, optimizer=opt, variational_state=vstate)

vmc.run(n_iter=300)
print("VMC energy:", vstate.expect(H).mean)



**What to check:**
- The VMC energy approaches the ED value within reasonable tolerance.
- Increasing `n_samples` or `n_iter` should reduce variance.

## Baseline 2D: small grid sanity check
The 2D TFIM is qualitatively different and quickly becomes intractable for ED. A tiny grid still lets us verify the operator and optimization behavior before scaling up.


A small 2D lattice provides a second baseline. ED will be limited to very small grids.



In [ ]:
L = 3
J = 1.0
h = 1.0

grid = nk.graph.Grid(extent=(L, L), pbc=True)
hilbert, H = make_tfim(grid, J=J, h=h)

E_ed = nk.exact.lanczos_ed(H, k=1, compute_eigenvectors=False)[0]
print("ED ground-state energy:", E_ed)

model = nk.models.RBM(alpha=2)
sa = nk.sampler.MetropolisLocal(hilbert, n_chains=32)
vstate = nk.vqs.MCState(sa, model, n_samples=4096, n_discard_per_chain=200)

opt = nk.optimizer.Sgd(learning_rate=0.05)
vmc = nk.VMC(hamiltonian=H, optimizer=opt, variational_state=vstate)

vmc.run(n_iter=400)
print("VMC energy:", vstate.expect(H).mean)



**What to check:**
- Energies remain stable across multiple random seeds.
- Small 2D lattices match ED to within sampling error.

## Long-range interactions: power-law couplings
Power-law interactions appear in dipolar, Rydberg, and trapped-ion systems. They increase the connectivity of the Hamiltonian, which is a harder test for sampling and the ansatz.


To add long-range interactions, we define couplings that decay as a power law with distance. For small sizes, compare to ED; for larger sizes, compare trends against 1D or 2D baselines.



In [ ]:
def long_range_edges_1d(N, alpha=3.0, J=1.0):
    edges = []
    weights = []
    for i in range(N):
        for j in range(i + 1, N):
            r = min(abs(i - j), N - abs(i - j))
            edges.append((i, j))
            weights.append(-J / (r ** alpha))
    return edges, weights


def make_long_range_tfim_1d(N, alpha=3.0, J=1.0, h=1.0):
    hilbert = nk.hilbert.Spin(s=0.5, N=N)
    edges, weights = long_range_edges_1d(N, alpha=alpha, J=J)

    H = nk.operator.LocalOperator(hilbert)
    for (i, j), w in zip(edges, weights):
        H += w * (nk.operator.spin.sigmaz(hilbert, i) @ nk.operator.spin.sigmaz(hilbert, j))
    for i in range(N):
        H += -h * nk.operator.spin.sigmax(hilbert, i)

    return hilbert, H



**What to check:**
- For small `N`, ED should match VMC to within sampling error.
- As `alpha` decreases (longer-range), the energy landscape changes and VMC convergence may slow.

## Scaling up: larger systems and stronger ansatz
As N grows, the variational manifold must capture more entanglement and correlations. Energy variance is a useful proxy: lower variance generally indicates a better approximation to an eigenstate.


Once baselines are validated, scale up by:

- Increasing `N` (1D) or `L` (2D).
- Raising `alpha` (more expressive RBM) or switching to a deeper model.
- Monitoring the energy variance as a proxy for ansatz quality.



In [ ]:
N = 32
chain = nk.graph.Chain(length=N, pbc=True)
hilbert, H = make_tfim(chain, J=1.0, h=1.0)

model = nk.models.RBM(alpha=4)
sa = nk.sampler.MetropolisLocal(hilbert, n_chains=64)
vstate = nk.vqs.MCState(sa, model, n_samples=8192, n_discard_per_chain=200)

opt = nk.optimizer.Sgd(learning_rate=0.03)
vmc = nk.VMC(hamiltonian=H, optimizer=opt, variational_state=vstate)

vmc.run(n_iter=500)
print("VMC energy:", vstate.expect(H).mean)
print("Energy variance:", vstate.expect(H).variance)



## Results table (from my runs)

Environment: NetKet 3.21.0, RBM alpha=2, VMC with SGD, `n_samples=2048` (1D) / `4096` (2D, long-range), `n_iter=250–300`.

| System | J | h | Method | Energy | Notes |
| --- | --- | --- | --- | --- | --- |
| 1D, N=12 | 1.0 | 1.0 | ED | -15.3226 | baseline |
| 1D, N=12 | 1.0 | 1.0 | VMC | -15.2879 | RBM alpha=2, var≈0.24 |
| 2D, L=3 | 1.0 | 1.0 | ED | -12.0199 | baseline |
| 2D, L=3 | 1.0 | 1.0 | VMC | -11.9999 | RBM alpha=2, var≈0.10 |
| 1D, N=12, alpha=3 | 1.0 | 1.0 | ED | -16.8684 | long-range |
| 1D, N=12, alpha=3 | 1.0 | 1.0 | VMC | -16.8506 | RBM alpha=2, var≈0.11 |

**Quick read of the results:** the RBM ansatz tracks ED within ~0.02–0.035 energy units on these small systems. The 2D case reaches a lower variance with the same ansatz size, while the long-range case converges but stays slightly above ED. This is a reasonable baseline: increasing `alpha`, `n_samples`, or `n_iter` should tighten the gap, especially for long-range couplings. As always with VMC, stochastic noise means these numbers can shift with different random seeds and sampling budgets.

## Open questions to explore

- Where does the RBM ansatz begin to fail as range increases or near criticality?
- How do optimization hyperparameters scale with system size?
- Can symmetry constraints or alternative models reduce variance at large `N`?
- Does sampling mix poorly for certain `h/J` regimes, and how can we detect it early?

## Suggested next steps

- Add plots for energy vs `h/J` and energy variance vs `N`.
- Compare with a second ansatz (e.g., deeper neural network or autoregressive model).
- Explore a small 2D long-range case and note qualitative differences.

## Plots: energy vs h/J and variance vs N

To make the workflow tangible, we plot two diagnostics:

- **Energy vs h/J (ED vs VMC)** on a small 1D chain. This shows whether VMC tracks the exact curve across the phase transition.
- **Energy variance vs N (VMC only)** at fixed h/J. This gives a coarse view of how ansatz quality scales.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import netket as nk

def run_vmc(H, hilbert, alpha=2, n_iter=150, n_samples=1024, lr=0.05):
    model = nk.models.RBM(alpha=alpha)
    sa = nk.sampler.MetropolisLocal(hilbert, n_chains=32)
    vstate = nk.vqs.MCState(sa, model, n_samples=n_samples, n_discard_per_chain=100)
    opt = nk.optimizer.Sgd(learning_rate=lr)
    vmc = nk.VMC(hamiltonian=H, optimizer=opt, variational_state=vstate)
    vmc.run(n_iter=n_iter)
    stats = vstate.expect(H)
    return stats.mean, stats.variance

print('NetKet', nk.__version__)


In [ ]:
J = 1.0
N = 10
h_values = np.linspace(0.5, 1.5, 7)

chain = nk.graph.Chain(length=N, pbc=True)
ed_energies = []
vmc_energies = []

for h in h_values:
    hilbert, H = make_tfim(chain, J=J, h=h)
    E_ed = nk.exact.lanczos_ed(H, k=1, compute_eigenvectors=False)[0]
    E_vmc, _ = run_vmc(H, hilbert, alpha=2, n_iter=150, n_samples=1024, lr=0.05)
    ed_energies.append(E_ed / N)
    vmc_energies.append(E_vmc / N)

plt.figure(figsize=(6, 4))
plt.plot(h_values / J, ed_energies, 'o-', label='ED (per site)')
plt.plot(h_values / J, vmc_energies, 's--', label='VMC (RBM alpha=2)')
plt.xlabel('h/J')
plt.ylabel('Energy per site')
plt.title('1D TFIM: Energy vs h/J (N=10)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
h = 1.0
N_values = [8, 10, 12]
variances = []

for N in N_values:
    chain = nk.graph.Chain(length=N, pbc=True)
    hilbert, H = make_tfim(chain, J=J, h=h)
    _, var = run_vmc(H, hilbert, alpha=2, n_iter=150, n_samples=1024, lr=0.05)
    variances.append(var / N)

plt.figure(figsize=(6, 4))
plt.plot(N_values, variances, 'o-')
plt.xlabel('N')
plt.ylabel('Energy variance per site')
plt.title('1D TFIM: VMC variance vs N (h/J=1)')
plt.tight_layout()
plt.show()
